# Setup

## Imports

In [1]:
import magnet as mag

Accelerating your code on shiney new Tesla P100-PCIE-16GB GPU.


In [2]:
import torch

In [3]:
from captioner.eval import caption, pretty_print
from captioner.extract import Extractor
from captioner.hparams import (image_shape, architecture, hidden_size, num_layers,
                               rnn_type, vocab_size, beam_size, image_shape, probabilistic)
from captioner.model import Model
from captioner.nlp import get_nlp

In [4]:
from PIL import Image

## Preparations

In [5]:
DIR_DATA /= 'COCO'
DIR_CHECKPOINTS = DIR_MAIN / 'checkpoints'

filenames = list(DIR_DATA.glob('val/*.jpg'))

Load the spaCy model

In [6]:
nlp = get_nlp('en_core_web_lg', vocab_size, DIR_CHECKPOINTS / 'vocab')

embedding_dim = nlp.vocab.vectors.shape[1]

Get the extractor and it's feature size

In [7]:
extractor = Extractor(architecture)
feature_dim = extractor.feature_size

Load the model and the trained weights

In [8]:
model = Model(feature_dim, embedding_dim, hidden_size,
              num_layers, rnn_type, vocab_size)

device = 'cuda:0' if mag.device == 'cuda' else mag.device
model.load_state_dict(torch.load(DIR_CHECKPOINTS / 'model.pt', map_location=device))

# Caption

In [ ]:
img = Image.open(np.random.choice(filenames))
pretty_print(caption(img, model, extractor, nlp, beam_size, probabilistic, image_shape))
img